In [2]:
# -*- coding: utf-8 -*-
import re
import time
import random
from datetime import datetime

import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

# --------- Paramètres ---------
TICKERS = [
    "ABJC","BICB","BICC","BNBC","BOAB","BOABF","BOAC","BOAM","BOAN","BOAS",
    "CABC","CBIBF","CFAC","CIEC","ECOC","ETIT","FTSC","LNBB","NEIC","NSBC",
    "NTLC","ONTBF","ORAC","ORGT","PALC","PRSC","SAFC","SCRC","SDCC","SDSC",
    "SEMC","SGBC","SHEC","SIBC","SICC","SIVC","SLBC","SMBC","SNTS","SOGC",
    "SPHC","STAC","STBC","SVOC","TTLC","TTLS","UNLC","UNXC"
]

BASE_URL   = "https://www.richbourse.com/investisseur/rapport-activite/index/{ticker}/{type_}/{periode}"
AUTRES_URL = "https://www.richbourse.com/investisseur/analyse-societe/rapports-autres/{ticker}"

# --------- Helpers ---------
def clean_cell(val: str) -> str:
    if not val:
        return ""
    # coupe la variation (%) et nettoie espaces/virgules
    val = re.split(r"\(", val)[0]
    val = (val.replace("\xa0", " ")
              .replace(" ", "")
              .replace(",", ""))
    return val.strip()

def detect_unit_footer(soup: BeautifulSoup) -> str:
    patterns = [
        r"donn[ée]es?.*xof",
        r"donn[ée]es?.*francs?\s*cfa",
        r"donn[ée]es?.*milliers.*xof",
        r"en\s+milliers\s+de\s+xof"
    ]
    # on balaie pas mal d'éléments de fin de page
    for el in soup.select("p, small, div, span, footer")[::-1]:
        txt = el.get_text(" ", strip=True)
        if not txt:
            continue
        for pat in patterns:
            if re.search(pat, txt, flags=re.I):
                return txt
    return ""

def period_label_from_type(type_: str, periode: str) -> str:
    t = type_.lower()
    if t.startswith("trimes"):
        return f"T{periode}"
    if t.startswith("semest"):
        return "S1" if periode == "1" else "S2"
    return "An"

def period_label_from_text(txt: str) -> str:
    """Devine S1/S2/T1..T4/An à partir d'un libellé libre (rapports-autres)."""
    s = txt.lower()
    # trimestre
    m = re.search(r"(\bt\s*([1-4])\b|\btrimes?tre\s*([1-4])\b|\b([1-4])\s*e?r?\s*trimes?tre\b)", s)
    if m:
        # capture du numéro
        for g in m.groups()[1:]:
            if g and g.isdigit():
                return f"T{g}"
    # semestre
    m = re.search(r"semestre\s*([12])|\b([12])\s*e?r?\s*semestre\b|\bs([12])\b", s)
    if m:
        for g in m.groups():
            if g and g.isdigit():
                return f"S{g}"
    return "An"

def firstcol_contains_year(text: str, year: int) -> bool:
    return str(year) in (text or "")

def extract_headers_and_rows(table) -> tuple[list[str], list[list[str]]]:
    thead = table.find("thead")
    if thead:
        headers = [th.get_text(" ", strip=True) for th in thead.find_all("th")]
    else:
        first_tr = table.find("tr")
        headers = [td.get_text(" ", strip=True) for td in first_tr.find_all(["td","th"])]
    rows = []
    for tr in table.find_all("tr"):
        tds = [td.get_text(" ", strip=True) for td in tr.find_all("td")]
        if tds:
            rows.append(tds)
    return headers, rows

# --------- Scrapers ---------
async def scrape_report(page, ticker: str, type_: str, periode: str) -> pd.DataFrame:
    """Annuel/Semestriel/Trimestriel (tableau principal)."""
    url = BASE_URL.format(ticker=ticker, type_=type_, periode=periode)
    await page.goto(url, timeout=60000)

    try:
        await page.wait_for_selector("table.table", timeout=8000)
    except:
        return pd.DataFrame()

    html_page = await page.content()
    soup_page = BeautifulSoup(html_page, "html.parser")
    table = soup_page.select_one("table.table")
    if table is None:
        return pd.DataFrame()

    # entêtes
    thead = table.find("thead")
    if thead:
        headers = [th.get_text(strip=True) for th in thead.find_all("th")]
    else:
        first_tr = table.find("tbody").find("tr")
        headers = [td.get_text(strip=True) for td in first_tr.find_all("td")]

    if "Période" not in headers:
        headers = ["Période"] + headers

    # lignes
    rows = []
    body = table.find("tbody")
    if body:
        for tr in body.find_all("tr"):
            tds = [td.get_text(" ", strip=True) for td in tr.find_all("td")]
            if not tds:
                continue
            if len(tds) == len(headers) - 1 and headers[0] == "Période":
                tds = [""] + tds  # sécu si la période n’est pas rendue
            row = {headers[i]: clean_cell(tds[i]) for i in range(min(len(headers), len(tds)))}
            rows.append(row)

    df = pd.DataFrame(rows)
    if df.empty:
        return df

    # unité
    unit_phrase = detect_unit_footer(soup_page)

    # colonnes additionnelles
    type_norm = "Annuel" if type_.lower().startswith("annuel") else ("Semestriel" if type_.lower().startswith("semest") else "Trimestriel")
    df["Ticker"] = ticker
    df["Type"] = type_norm
    df["Période_détail"] = period_label_from_type(type_norm, periode)
    df["en XOF"] = unit_phrase

    # ordre
    front = ["Ticker", "Type", "Période_détail", "Période", "en XOF"]
    others = [c for c in df.columns if c not in front]
    return df[front + others]

async def scrape_rapports_autres(page, ticker: str) -> pd.DataFrame:
    """
    /rapports-autres/{ticker}
    - Conserve uniquement les lignes dont la 1ʳᵉ cellule contient l'année courante
    - Devine Période_détail (S1/S2/T1..T4/An) depuis le libellé de la 1ʳᵉ cellule
    - Période = année détectée (courante)
    - en XOF = note sous le 1er tableau (fallback: détection globale)
    """
    url = AUTRES_URL.format(ticker=ticker)
    await page.goto(url, timeout=60000)

    try:
        await page.wait_for_selector("table", timeout=8000)
    except:
        return pd.DataFrame()

    html = await page.content()
    soup = BeautifulSoup(html, "html.parser")
    tables = soup.select("table")
    if not tables:
        return pd.DataFrame()

    current_year = datetime.now().year

    # Unité : d’abord l’élément suivant le 1er tableau, sinon fallback global
    unit_phrase = ""
    first_table = tables[0]
    sib = first_table.find_next_sibling()
    while sib and sib.name in ["table","script","style"]:
        sib = sib.find_next_sibling()
    if sib:
        txt = sib.get_text(" ", strip=True)
        if re.search(r"(donn[ée]es?|en)\s+.*(xof|francs?\s*cfa|milliers)", txt, flags=re.I):
            unit_phrase = txt
    if not unit_phrase:
        unit_phrase = detect_unit_footer(soup)

    collected = []
    for table in tables:
        headers, rows = extract_headers_and_rows(table)
        if not headers or not rows:
            continue

        if "Période" not in headers:
            headers = ["Période"] + headers

        for r in rows:
            # force une colonne "Période" si absente
            if len(r) == len(headers) - 1:
                r = [""] + r
            if not r:
                continue
            first_cell_raw = r[0]
            if not firstcol_contains_year(first_cell_raw, current_year):
                continue

            row_dict = {headers[i]: clean_cell(r[i]) for i in range(min(len(headers), len(r)))}
            # Ajouts format final
            row_dict["Ticker"] = ticker
            row_dict["Type"] = "Autres"
            row_dict["Période_détail"] = period_label_from_text(first_cell_raw)
            row_dict["Période"] = str(current_year)
            row_dict["en XOF"] = unit_phrase
            collected.append(row_dict)

    if not collected:
        return pd.DataFrame()

    df = pd.DataFrame(collected)
    front = ["Ticker", "Type", "Période_détail", "Période", "en XOF"]
    others = [c for c in df.columns if c not in front]
    return df[front + others]

# --------- Orchestrateur ---------
async def main(output_csv: str = "rapport_brvm_complet.csv", headless: bool = True):
    all_parts = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=headless)
        try:
            context = await browser.new_context(storage_state="cookies.json")
        except:
            context = await browser.new_context()
        page = await context.new_page()

        for ticker in TICKERS:
            print(f"📊 {ticker} ...")
            dfs = []
            # Annuel
            dfs.append(await scrape_report(page, ticker, "Annuel", "1"))
            # Semestriels
            dfs.append(await scrape_report(page, ticker, "Semestriel", "1"))
            dfs.append(await scrape_report(page, ticker, "Semestriel", "2"))
            # Trimestriels
            for i in range(1, 4+1):
                dfs.append(await scrape_report(page, ticker, "Trimestriel", str(i)))
            # Rapports "autres" (filtrés sur l'année courante)
            dfs.append(await scrape_rapports_autres(page, ticker))

            part = pd.concat([d for d in dfs if isinstance(d, pd.DataFrame) and not d.empty],
                             ignore_index=True) if dfs else pd.DataFrame()

            if part.empty:
                all_parts.append(pd.DataFrame([{
                    "Ticker": ticker, "Type": "Aucune donnée",
                    "Période_détail": "", "Période": "", "en XOF": ""
                }]))
            else:
                all_parts.append(part)

            time.sleep(0.5 + random.random()*0.7)

        await browser.close()

    df_final = pd.concat(all_parts, ignore_index=True) if all_parts else pd.DataFrame()
    if df_final.empty:
        print("⚠️ Aucun rapport récupéré")
        return df_final

    first_cols = ["Ticker", "Type", "Période_détail", "Période", "en XOF"]
    df_final = df_final[first_cols + [c for c in df_final.columns if c not in first_cols]]
    df_final.to_csv(output_csv, index=False)
    print(f"✅ Fichier {output_csv} généré ({len(df_final)} lignes)")
    return df_final

# --- Exécution dans Jupyter ---
df = await main(headless=False)
# df.head()


📊 ABJC ...
📊 BICB ...
📊 BICC ...
📊 BNBC ...
📊 BOAB ...
📊 BOABF ...
📊 BOAC ...
📊 BOAM ...
📊 BOAN ...
📊 BOAS ...
📊 CABC ...
📊 CBIBF ...
📊 CFAC ...
📊 CIEC ...
📊 ECOC ...
📊 ETIT ...
📊 FTSC ...
📊 LNBB ...
📊 NEIC ...
📊 NSBC ...
📊 NTLC ...
📊 ONTBF ...
📊 ORAC ...
📊 ORGT ...
📊 PALC ...
📊 PRSC ...
📊 SAFC ...
📊 SCRC ...
📊 SDCC ...
📊 SDSC ...
📊 SEMC ...
📊 SGBC ...
📊 SHEC ...
📊 SIBC ...
📊 SICC ...
📊 SIVC ...
📊 SLBC ...
📊 SMBC ...
📊 SNTS ...
📊 SOGC ...
📊 SPHC ...
📊 STAC ...
📊 STBC ...
📊 SVOC ...
📊 TTLC ...
📊 TTLS ...
📊 UNLC ...
📊 UNXC ...
✅ Fichier rapport_brvm_complet.csv généré (1585 lignes)


In [3]:
import pandas as pd
import numpy as np



# Charger le fichier
df = pd.read_csv("rapport_brvm_complet.csv")

# --- Uniformisation en XOF (reprend la logique précédente, en vectorisé) ---
def get_multiplier(unit: str) -> int:
    if pd.isna(unit):
        return 1
    unit = str(unit).lower()
    if "milliers" in unit:
        return 1_000
    elif "millions" in unit:
        return 1_000_000
    else:
        return 1

# Colonnes numériques à corriger (à partir de la 6e jusqu'à la fin)
num_cols = df.columns[5:]

# Calcul vectorisé du facteur
factors = df["en XOF"].map(get_multiplier)

# Conversion sûre et multiplication
df[num_cols] = df[num_cols].apply(
    lambda s: pd.to_numeric(s.replace("-", np.nan), errors="coerce") * factors
)

# Unités uniformisées
df["en XOF"] = "XOF"

# --- Ops demandées ---
# 1) Supprimer la colonne "Type" si elle existe
if "Type" in df.columns:
    df = df.drop(columns=["Type"])

# 2) "Période" -> 4 derniers caractères (année)
if "Période" in df.columns:
    annee = (
        df["Période"].astype(str).str[-4:].str.extract(r"(\d{4})")[0].astype("Int64")
    )
    df["Période"] = annee

# --- Déduplication avancée sur (Ticker, Période_détail, Période) en gardant la ligne la plus complète ---
subset = ["Ticker", "Période_détail", "Période"]
missing_subset = [c for c in subset if c not in df.columns]
if missing_subset:
    raise KeyError(f"Colonnes manquantes pour la déduplication: {missing_subset}")

# Compter "l'information" présente par ligne (champ non vide, non '-', non NaN)
# Sans modifier df: on crée une copie normalisée pour le scoring
tmp = df.replace(
    to_replace=[r"^\s*$", r"^-+$", r"(?i)^nan$"],
    value=pd.NA,
    regex=True
)

# Score global de complétude
score_all = tmp.notna().sum(axis=1)

# Score numérique (optionnel, utile comme briseur d'égalité)
numeric_cols = [c for c in df.columns if c in num_cols]
score_num = tmp[numeric_cols].notna().sum(axis=1) if numeric_cols else pd.Series(0, index=df.index)

# Créer une clé de tri pour choisir la "meilleure" ligne dans chaque groupe
df["__score_all__"] = score_all
df["__score_num__"] = score_num

# Identifier les doublons (groupes ayant au moins 2 lignes)
dupe_groups = df.duplicated(subset=subset, keep=False)
nb_groups = (
    df.loc[dupe_groups]
      .drop_duplicates(subset=subset)
      .shape[0]
)

print(f"🔎 Groupes en doublon sur {tuple(subset)} : {nb_groups}")

# Garder, pour chaque groupe, l'index de la ligne avec le score max (puis score_num en tie-break)
# On tri d’abord par scores desc, puis on prend le premier de chaque groupe
df_sorted = df.sort_values(by=["__score_all__", "__score_num__"], ascending=False)
best_idx = df_sorted.groupby(subset, dropna=False, as_index=False).head(1).index

# Conserver ces lignes "meilleures" + toutes les lignes uniques (non en doublon)
df_dedup = pd.concat([
    df.loc[~dupe_groups],     # uniques
    df.loc[best_idx]          # meilleurs des groupes
], ignore_index=True)

# Supprimer d’éventuels doublons résiduels exacts et nettoyer colonnes techniques
df_dedup = df_dedup.drop(columns=["__score_all__", "__score_num__"], errors="ignore")
df_dedup = df_dedup.drop_duplicates().reset_index(drop=True)

# Export
df_dedup.to_csv("rapport_brvm_complet_uniformise.csv", index=False)
# Export en Excel
df_dedup.to_excel("rapport_brvm_complet_uniformise.xlsx", index=False)

print("✅ Doublons (Ticker, Période_détail, Période) résolus en gardant la ligne la plus complète.")
print("✅ Export réalisé dans rapport_brvm_complet_uniformise.csv")


🔎 Groupes en doublon sur ('Ticker', 'Période_détail', 'Période') : 49
✅ Doublons (Ticker, Période_détail, Période) résolus en gardant la ligne la plus complète.
✅ Export réalisé dans rapport_brvm_complet_uniformise.csv


In [4]:
import pandas as pd
import numpy as np

# Charger le CSV
df = pd.read_csv("rapport_brvm_complet_uniformise.csv")

# Colonnes utiles
colonnes_utiles = [
    "Ticker", "Période_détail", "Période",
    "Chiffre d'Affaires", "Produit net bancaire",
    "Total Bilan",
    "Résultat d'Exploitation", "Résultat Brut d'Exploitation",
    "Résultat Net"
]
df = df[colonnes_utiles]

# --- Liste complète des banques cotées à la BRVM ---
banques = [
    "BICB","BICC","BOAB","BOABF","BOAC","BOAM","BOAN","BOAS",
    "CBIBF","ECOC","NSBC","SGBC","SIBC","ETIT"  # <-- inclut ETIT
]

# --- Colonnes calculées ---
df["CA_PNB"] = np.where(
    df["Ticker"].isin(banques),
    df["Produit net bancaire"],
    df["Chiffre d'Affaires"]
)

df["RE_RBE"] = np.where(
    df["Ticker"].isin(banques),
    df["Résultat Brut d'Exploitation"],
    df["Résultat d'Exploitation"]
)

# Ajouter une colonne Capitaux propres vide
df["Capitaux propres"] = None

# Colonnes finales
df_final = df[[
    "Ticker", "Période_détail", "Période",
    "CA_PNB", "Total Bilan", "RE_RBE", "Résultat Net", "Capitaux propres"
]]

# Export
df_final.to_csv("rapport_brvm_complet_uniformise2.csv", index=False)

print("✅ Fichier prêt : rapport_brvm_complet_uniformise2.csv")


✅ Fichier prêt : rapport_brvm_complet_uniformise2.csv


In [5]:
import re
import pandas as pd
import numpy as np
from itertools import product

# Fichiers
SRC = "rapport_brvm_complet_uniformise2.csv"
DST = "rapport_brvm_complet_uniformise3.csv"

# Variables à pivoter
vars_keep = ["CA_PNB", "Total Bilan", "RE_RBE", "Résultat Net", "Capitaux propres"]

# Charge
df = pd.read_csv(SRC)

# Vérifications
cols_needed = ["Ticker","Période_détail","Période"] + vars_keep
missing = [c for c in cols_needed if c not in df.columns]
if missing:
    raise ValueError(f"Colonnes manquantes dans {SRC}: {missing}")

# Dédupe
df = df.sort_values(["Période_détail","Période","Ticker"]).drop_duplicates(
    subset=["Période_détail","Période","Ticker"], keep="first"
)

# Normalise "Période" -> année numérique
def extract_year(x):
    s = str(x)
    m = re.search(r"\d{4}", s)
    return int(m.group(0)) if m else np.nan

df["__Year"] = df["Période"].apply(extract_year)

# Range d'années: 1998 -> max année détectée (au moins 1998)
year_min = 1998
year_max = int(np.nanmax(df["__Year"])) if df["__Year"].notna().any() else 1998
years = list(range(year_min, year_max + 1))

# Nouvel ordre souhaité: T1..T4, S1..S2, puis An
ordre_pd = ["T1","T2","T3","T4","S1","S2","An"]
df["__PeriodeOrd"] = pd.Categorical(df["Période_détail"], categories=ordre_pd, ordered=True)

# Tickers
tickers = sorted(df["Ticker"].unique())

# Index complet (Période_détail, Year, Ticker)
full_index = pd.MultiIndex.from_tuples(
    list(product(ordre_pd, years, tickers)),
    names=["Période_détail", "__Year", "Ticker"]
)

# Réindexe pour compléter les manquants
base = df.set_index(["Période_détail","__Year","Ticker"])
base = base[vars_keep].reindex(full_index)

# Reconstruit "Période" depuis l'année
base = base.reset_index()
base["Période"] = base["__Year"].astype("Int64").astype(str)

# Pivot large: colonnes "Ticker_Variable"
base = base.set_index(["Période_détail","Période","Ticker"])
pieces = []
for v in vars_keep:
    w = base[v].unstack("Ticker")      # colonnes=Tickers
    w = w.add_suffix(f"_{v}")          # "ABJC_CA_PNB", etc.
    pieces.append(w)

wide = pd.concat(pieces, axis=1).reset_index()

# Tri: année croissante puis ordre custom T→S→An
def extract_year_safe(x):
    try:
        return int(re.search(r"\d{4}", str(x)).group(0))
    except:
        return np.nan

wide["__Year"] = wide["Période"].apply(extract_year_safe)
wide["__PeriodeOrd"] = pd.Categorical(wide["Période_détail"], categories=ordre_pd, ordered=True)
wide = wide.sort_values(["__Year","__PeriodeOrd","Période_détail","Période"], na_position="last")

# --- NOUVEAU : réordonner les colonnes par Ticker puis variables ---
fixed_prefix = ["Période_détail", "Période"]
candidate = [f"{t}_{v}" for t in tickers for v in vars_keep]
ordered_data_cols = [c for c in candidate if c in wide.columns]  # garde l'ordre souhaité, ignore si colonne manquante
other_cols = [c for c in wide.columns if c not in fixed_prefix + ordered_data_cols + ["__Year","__PeriodeOrd"]]
wide = wide[fixed_prefix + ordered_data_cols + other_cols]
# -------------------------------------------------------------------

# Nettoyage
wide = wide.drop(columns=["__Year","__PeriodeOrd"], errors="ignore")

# Export
wide.to_csv(DST, index=False)
print(f"✅ Pivot complété & trié (T→S→An) et colonnes groupées par ticker exporté : {DST}")


✅ Pivot complété & trié (T→S→An) et colonnes groupées par ticker exporté : rapport_brvm_complet_uniformise3.csv


In [6]:
import pandas as pd

SRC = "rapport_brvm_complet_uniformise3.csv"

AN_DST = "rapport_brvm_Annuel.csv"
TRI_DST = "rapport_brvm_Trimestriel.csv"
SEM_DST = "rapport_brvm_Semestriel.csv"

XLSX_DST = "rapport_brvm_decoupe.xlsx"

# charge la table pivot large
df = pd.read_csv(SRC)

# garde uniquement les colonnes attendues au cas où
assert "Période_détail" in df.columns, "Colonne 'Période_détail' manquante dans le fichier source."

# filtres
annuel = df[df["Période_détail"] == "An"].copy()
trimestriel = df[df["Période_détail"].isin(["T1","T2","T3","T4"])].copy()
semestriel = df[df["Période_détail"].isin(["S1","S2"])].copy()

# export CSV
annuel.to_csv(AN_DST, index=False)
trimestriel.to_csv(TRI_DST, index=False)
semestriel.to_csv(SEM_DST, index=False)

# export Excel avec 3 onglets
with pd.ExcelWriter(XLSX_DST, engine="openpyxl") as writer:
    annuel.to_excel(writer, sheet_name="Annuel", index=False)
    trimestriel.to_excel(writer, sheet_name="Trimestriel", index=False)
    semestriel.to_excel(writer, sheet_name="Semestriel", index=False)

print("✅ Fichiers créés :")
print(f"- {AN_DST}")
print(f"- {TRI_DST}")
print(f"- {SEM_DST}")
print(f"- {XLSX_DST} (avec 3 onglets)")


✅ Fichiers créés :
- rapport_brvm_Annuel.csv
- rapport_brvm_Trimestriel.csv
- rapport_brvm_Semestriel.csv
- rapport_brvm_decoupe.xlsx (avec 3 onglets)
